In [4]:
import pandas as pd
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

True

In [ ]:
# Get paths from environment variables
base_path = os.getenv('HTML_PATH')
download_path = os.getenv('DOWNLOAD_PATH')

# Construct full path to HTML file
path = os.path.join(base_path, "3.htm")

HTML file path: C:\Users\calli\Downloads\PROP-database-new\da_aggiungere\fnext\100\f\3.htm
Download directory: C:\Users\calli\Downloads\PROP-database-new\da_aggiungere\fnext\100\f


In [6]:
# Check if file exists before reading
if not os.path.exists(path):
    print(f"Error: HTML file not found at {path}")
else:
    # Read HTML file
    tables = pd.read_html(path)
    
    # Check if tables are found
    if not tables:
        print("No tables found in the HTML file.")
    else:
        print(f"Found {len(tables)} tables in the HTML file.")
        for i, table in enumerate(tables, 1):
            print(f"Table {i}: {table.shape[0]} rows x {table.shape[1]} columns")

Found 1 tables in the HTML file.
Table 1: 673 rows x 14 columns


In [ ]:
def html_to_excel(html_file, excel_file):
    try:
        # Read HTML tables from the file
        tables = pd.read_html(html_file)
        if not tables:
            print("No tables found in the HTML file.")
            return
        
        # Save each table to a separate sheet in Excel
        with pd.ExcelWriter(excel_file, engine='openpyxl') as writer:
            for idx, table in enumerate(tables, start=1):
                sheet_name = f"Sheet{idx}"
                table.to_excel(writer, sheet_name=sheet_name, index=False)
        
        print(f"Conversion successful!")
    
    except FileNotFoundError:
        print(f"Error: File '{html_file}' not found.")
    except ValueError as e:
        print(f"Error reading HTML: {e}")
    except Exception as e:
        print(f"Unexpected error: {e}")

# Convert the HTML file to Excel
if os.path.exists(path):
    excel_output = os.path.join(download_path, "converted_tables.xlsx")
    html_to_excel(path, excel_output)
else:
    print("Cannot convert: HTML file not found.")

Conversion successful! Saved to 'C:\Users\calli\Downloads\PROP-database-new\da_aggiungere\fnext\100\f\converted_tables.xlsx'


In [24]:
# read converted csv file
df = pd.read_excel(excel_output, sheet_name='Sheet1')
df.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,Account: 55016647,Account: 55016647,Name: FundedNext-STLR 1-Step Account- Simone C...,Name: FundedNext-STLR 1-Step Account- Simone C...,Name: FundedNext-STLR 1-Step Account- Simone C...,Name: FundedNext-STLR 1-Step Account- Simone C...,Name: FundedNext-STLR 1-Step Account- Simone C...,Currency: USD,Currency: USD,Leverage:,Leverage:,"2025 November 2, 10:46","2025 November 2, 10:46","2025 November 2, 10:46"
1,Closed Transactions:,Closed Transactions:,Closed Transactions:,Closed Transactions:,Closed Transactions:,Closed Transactions:,Closed Transactions:,Closed Transactions:,Closed Transactions:,Closed Transactions:,Closed Transactions:,Closed Transactions:,Closed Transactions:,NaN
2,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price,Commission,Taxes,Swap,Profit
3,3891066,2025.10.31 03:00:01,buy,0.03,ndx100,26032.18,25794.45,26278.24,2025.10.31 19:58:51,25794.23,0.00,0.00,0.00,-71.39
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223041,223041[sl],223041[sl],223041[sl],NaN


In [25]:
# keep rows from row 2
df = df.iloc[2:].reset_index(drop=True)

# RangeIndex have the corrected column names
df.columns = df.iloc[0]
df = df[1:]

# trim whitespace from column names
df.columns = df.columns.str.strip()

In [ ]:
"""
drop rows where 'Open Time' and 'Close Time' have the following values:
Closed P/L:
Open Trades:
Open Time
Working Orders:
No transactions
"""
# drop strings in 'Open Time' and 'Close Time' columns
strings_to_drop = ['Closed P/L:', 'Open Trades:', 'Open Time', 'Working Orders:', 'No transactions']
df = df[~df['Open Time'].isin(strings_to_drop)]
df = df[~df['Close Time'].isin(strings_to_drop)]

In [28]:
# convert Open Time and Close Time to datetime
df['Open Time'] = pd.to_datetime(df['Open Time'])
df['Close Time'] = pd.to_datetime(df['Close Time'])

ValueError: time data "Closed P/L:" doesn't match format "%Y.%m.%d %H:%M:%S", at position 181. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [26]:
df.head()

,Ticket,Open Time,Type,Size,Item,Price,S / L,T / P,Close Time,Price,Commission,Taxes,Swap,Profit
1,3891066,2025.10.31 03:00:01,buy,0.03,ndx100,26032.18,25794.45,26278.24,2025.10.31 19:58:51,25794.23,0.00,0.00,0.00,-71.39
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,223041,223041[sl],223041[sl],223041[sl],NaN
3,3891482,2025.10.31 05:00:00,buy stop,0.04,ndx100,26235.72,26078.31,0.00,2025.10.31 15:00:00,26080.06,cancelled,cancelled,cancelled,cancelled
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,149,cancelled,cancelled,cancelled,NaN
5,3891485,2025.10.31 05:00:00,buy stop,0.04,ndx100,26235.72,26078.31,0.00,2025.10.31 09:00:00,26024.31,cancelled,cancelled,cancelled,cancelled


In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 670 entries, 1 to 670
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Ticket      351 non-null    object
 1   Open Time   351 non-null    object
 2   Type        351 non-null    object
 3   Size        351 non-null    object
 4   Item        351 non-null    object
 5   Price       351 non-null    object
 6   S / L       351 non-null    object
 7   T / P       351 non-null    object
 8   Close Time  349 non-null    object
 9   Price       661 non-null    object
 10  Commission  663 non-null    object
 11  Taxes       663 non-null    object
 12  Swap        663 non-null    object
 13  Profit      350 non-null    object
dtypes: object(14)
memory usage: 73.4+ KB
